# "Kalman Filtering"

> "Intution and high level concepts with code"

- toc: true
- branch: master
- badges: false
- comments: true
- categories: [Robotics]
- hide: false
- search_exclude: false
- image: images/post-thumbnails/robotics.jpg
- metadata_key1: Robotics
- metadata_key2: 

# Kalman Filter 

Its an iterative algorithm to reduce noise (or uncertainity) and arrive to a really good approximate value efficiently. 

Kalman filter has the these high level components:-

- **PREDICTION** - Predict the next state of the object. 
- **MEASUREMENT** - Measure the current state of the object.
- **CONFIDENCE** - How good (or bad) the PREDICTED or MEASUREMENT values

# Intuition

Kalman filter (or variants) have the following intuition.

**CURRENT STATE**

- Suppose current time is 3:00pm

**PREDICTION over MEASUREMENT**

- After few mins, we look at the watch again and we PREDICT its 3:04pm, but its shows 3:55pm!! 
- We know the MEASUREMENT is wrong and  have every reason to believe the PREDICTION is more acccurate and set the watch to be 3:04pm. Our CONFIDENCE in the MEASUREMENT is shaken, we believe it is so noisy to show 3:55pm 

**MEASUREMENT over PREDICTION**

- On the other hand, if the measurement shows 3:05 pm, we have greater confidence that MEASUREMENT is correct, since its more or less in line with our expectations and since its actually measured, thats should be more accurate. Here we trust or more CONFIDENCE in the MEASUREMENT. 

# Visual

![](https://abhisheksreesaila.github.io/blog/images/general/KF1.png "Kalman Filter")


# 3 Main Calculations


## Calculate the kalman gain

KG = $\frac {Eest} {Eest + Emea} $ ...(1)  0 <= KG <= 1

meaurements are more accurate if KG is close to 1
estimate are more correct if KG is close to 0

## Calculate the current estimate

EST(t) = EST(t-1) + KG [MEA - EST(t-1)]  ....(2)

ESTt-1 = Previous Estimate
ESTt = Current Estimate
MEA = MEasurement
Emea = Error in measurement

## Calculate the error in estimate

     Eest(t) = Emea * Eest(t-1) / Emea + Eest(t-1) ....(3)
===> Eest(t) = [1-KG] Eest(t-1)

What does this mean?

If measurement comes with a very little error, then we are closing in on the true value.  Emea is low, then KG is high (close to 1).
Then (1-KG) is very low. that means its drops the value of Eest(t-1) significantly, hence moving towards the "true" value.  Intuitively it makes sense, measurements are good, the system should converge to a good estimate.  Note that, in the real world, no matter how good the measurement is there is always noise, it will never be perfect. 

The opposite is also true. Measurements are unstable, then Emeais large, KG is low, 1-KG is high (or close to 1). Then we can trust the measurements coming, so we do not move the needle as much and try to keep the value close to the current error in estimate value as much as possible.  


Here is a fantastic explaination
https://www.youtube.com/watch?v=X9cC0o9viTo&list=PLX2gX-ftPVXU3oUFNATxGXY90AULiqnWT&index=4

Time for some code to explain and see how kalman filter can find the "true" value" with an example

# Code

In [ ]:
#hide
! pip install -U numpy
! pip install -U matplotlib

In [ ]:

import numpy as np

# TRUE VALUE. We will see how quickly and how close kalman filter can get
true_temp = 72

#=======================================================

#measurement read from the sensor
def readSensor():
    return np.random.randint(68,76)

#Initial Setting
initial_estimate = 68
intitial_error_in_estimate = 2
initial_measurement = 75
error_in_measurement = 4

#declare variable to be used later 
new_error_in_estimate = 0
current_estimate = 0

store_estimates_for_graph = []

#kalman filter iterations
duration = 20
time_elapsed = 0

while time_elapsed < duration:

  # if its first time, use these initial values else,
  if (time_elapsed == 0):
    error_in_estimate = intitial_error_in_estimate
    previous_estimate = initial_estimate
    measurement = initial_measurement
    
  else:
   # use calculated values from the prev iteration
    error_in_estimate = new_error_in_estimate
    previous_estimate = current_estimate
    measurement = readSensor()

 # 3 calculations from above 
  kalman_gain =  error_in_estimate/(error_in_estimate + error_in_measurement)
  current_estimate = previous_estimate + (kalman_gain * (measurement - previous_estimate))
  new_error_in_estimate = (1 - kalman_gain)*(error_in_estimate)

  print("New Estimate ->", str(current_estimate))
  store_estimates_for_graph.append(current_estimate)
  time_elapsed+=1


In [ ]:
import matplotlib.pyplot as plt

x = total_time = range(duration)
y = store_estimates_for_graph

plt.plot(x, y)  # Plot the chart

x1 = total_time 
y1 = [true_temp] * duration
  
# second plot with x1 and y1 data
plt.plot(x1, y1, '--')
 

plt.xlabel("time elapsed")
plt.ylabel("temperature")
plt.title('kalman filter')

plt.show()  # display

# References

The only and only Michel Van Biezen Explanation example for Kalman Filter
https://www.youtube.com/watch?v=PZrFFg5_Sd0&list=PLX2gX-ftPVXU3oUFNATxGXY90AULiqnWT&index=5
    